In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
#Plot Maps
import geopandas
import pycountry 
from ipywidgets import interact, IntSlider

from sklearn.impute import SimpleImputer
import datetime

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from functools import partial
import csv

import os

In [2]:
og = pd.read_csv('dataset/EMDAT_1900-2021_NatDis.csv')

In [3]:
og.loc[(og.Country == 'Bahamas (the)'),'Country']='Bahamas'
og.loc[(og.Country == 'Bolivia (Plurinational State of)'),'Country']='Bolivia, Plurinational State of'
og.loc[(og.Country == 'Comoros (the)'),'Country']='Comoros'
og.loc[(og.Country == 'Congo (the Democratic Republic of the)'),'Country']='Congo, the Democratic Republic of the'
og.loc[(og.Country == 'Congo (the)'),'Country']='Congo'
og.loc[(og.Country == 'Côte d’Ivoire'),'Country']="Côte d'Ivoire"
og.loc[(og.Country == 'Czech Republic (the)'),'Country']="Czechia"
og.loc[(og.Country == 'Dominican Republic (the)'),'Country']="Dominican Republic"
og.loc[(og.Country == 'Gambia (the)'),'Country']="Gambia"
og.loc[(og.Country == 'Iran (Islamic Republic of)'),'Country']="Iran, Islamic Republic of" 
og.loc[(og.Country == "Korea (the Democratic People's Republic of)"),'Country']="Korea, Democratic People's Republic of" 
og.loc[(og.Country == "Korea (the Republic of)"),'Country']="Korea, Republic of" 
og.loc[(og.Country == "Lao People's Democratic Republic (the)"),'Country']="Lao People's Democratic Republic" 
og.loc[(og.Country == "Macedonia (the former Yugoslav Republic of)"),'Country']="North Macedonia"
og.loc[(og.Country == "Marshall Islands (the)"),'Country']="Marshall Islands" 
og.loc[(og.Country == "Micronesia (Federated States of)"),'Country']="Micronesia, Federated States of" 
og.loc[(og.Country == "Moldova (the Republic of)"),'Country']="Moldova, Republic of" 
og.loc[(og.Country == "Netherlands (the)"),'Country']="Netherlands" 
og.loc[(og.Country == "Niger (the)"),'Country']="Niger" 
og.loc[(og.Country == "Philippines (the)"),'Country']="Philippines" 
og.loc[(og.Country == "Russian Federation (the)"),'Country']="Russian Federation" 
og.loc[(og.Country == "Sudan (the)"),'Country']="Sudan" 
og.loc[(og.Country == "Taiwan (Province of China)"),'Country']="Taiwan"
og.loc[(og.Country == "United Arab Emirates (the)"),'Country']="United Arab Emirates"
og.loc[(og.Country == "United Kingdom of Great Britain and Northern Ireland (the)"),'Country']="United Kingdom" 
og.loc[(og.Country == 'United States of America (the)'),'Country']="United States" 
og.loc[(og.Country == 'Venezuela (Bolivarian Republic of)'),'Country']='Venezuela, Bolivarian Republic of' 
og.loc[(og.Country == 'Swaziland'),'Country']='Eswatini'

In [4]:
og.head()

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Entry Criteria,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
0,1900-9002-CPV,1900,9002,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,...,NaN,11000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389
1,1900-9001-IND,1900,9001,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,...,NaN,1250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389
2,1902-0012-GTM,1902,12,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Kill,...,18.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,3.391845
3,1902-0003-GTM,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,...,8.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845
4,1902-0010-GTM,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,...,24.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845


In [5]:
cd = pd.read_csv('FoundCoords/Countries.csv', encoding='utf-16')

In [6]:
cd.head()

,Unnamed: 0,Country,CountryAlpha2,Count,Found Location,Found Latitude,Found Longitude
0,212,United States,US,1059,United States,39.783730,-100.445882
1,40,China,CN,970,中国,35.000074,104.999927
2,90,India,IN,740,India,22.351115,78.667743
3,155,Philippines,PH,659,Philippines,12.750349,122.731210
4,91,Indonesia,ID,560,Indonesia,-2.483383,117.890285


In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)

In [8]:
cd['Found Latitude'].isnull().sum()

21

In [9]:
cd.loc[(cd.Country == 'Hong Kong'), ['Found Latitude', 'Found Longitude']]=[22.352957, 113.962891]
cd.loc[(cd.Country == 'Korea, Republic of'), ['Found Latitude', 'Found Longitude']]=[36.638392, 127.6961188]
cd.loc[(cd.Country == 'Soviet Union'), ['Found Latitude', 'Found Longitude']]=[52.872309, 78.615662]
cd.loc[(cd.Country == 'Korea, Democratic People\'s Republic of'), ['Found Latitude', 'Found Longitude']]=[40.3736611, 127.0870417]
cd.loc[(cd.Country == 'Puerto Rico'), ['Found Latitude', 'Found Longitude']]=[18.2247706, -66.4858295]
cd.loc[(cd.Country == 'Yugoslavia'), ['Found Latitude', 'Found Longitude']]=[44.300170, 18.979612]
cd.loc[(cd.Country == 'Germany Fed Rep'), ['Found Latitude', 'Found Longitude']]=[50.626761, 9.037884]
cd.loc[(cd.Country == 'Martinique'), ['Found Latitude', 'Found Longitude']]=[14.6367927, -61.01582685063731]
cd.loc[(cd.Country == 'New Caledonia'), ['Found Latitude', 'Found Longitude']]=[-21.3019905, 165.4880773]
cd.loc[(cd.Country == 'Guadeloupe'), ['Found Latitude', 'Found Longitude']]=[16.2528827, -61.5686855]
cd.loc[(cd.Country == 'Moldova, Republic of'), ['Found Latitude', 'Found Longitude']]=[47.2879608, 28.5670941]
cd.loc[(cd.Country == 'Serbia Montenegro'), ['Found Latitude', 'Found Longitude']]=[43.821067, 20.632379]
cd.loc[(cd.Country == 'Réunion'), ['Found Latitude', 'Found Longitude']]=[-21.130737949999997, 55.536480112992315]
cd.loc[(cd.Country == 'Cook Islands (the)'), ['Found Latitude', 'Found Longitude']]=[-19.99697155, -157.78587140620786]
cd.loc[(cd.Country == 'Guam'), ['Found Latitude', 'Found Longitude']]=[13.450125700000001, 144.75755102972062]
cd.loc[(cd.Country == 'Czechoslovakia'), ['Found Latitude', 'Found Longitude']]=[49.323182, 17.621027]
cd.loc[(cd.Country == 'Yemen Arab Rep'), ['Found Latitude', 'Found Longitude']]=[15.429536, 44.818064]
cd.loc[(cd.Country == 'Palestine, State of'), ['Found Latitude', 'Found Longitude']]=[31.947953, 35.199782]
cd.loc[(cd.Country == 'French Polynesia'), ['Found Latitude', 'Found Longitude']]=[-17.0243749, -144.6434898]
cd.loc[(cd.Country == 'Virgin Island (U.S.)'), ['Found Latitude', 'Found Longitude']]=[17.789187, -64.7080574]
cd.loc[(cd.Country == 'Turks and Caicos Islands (the)'), ['Found Latitude', 'Found Longitude']]=[21.721746, -71.5527809]
cd.loc[(cd.Country == 'Macao'), ['Found Latitude', 'Found Longitude']]=[22.1899448, 113.5380454]
cd.loc[(cd.Country == 'Yemen P Dem Rep'), ['Found Latitude', 'Found Longitude']]=[16.120584, 48.932668]
cd.loc[(cd.Country == 'Canary Is'), ['Found Latitude', 'Found Longitude']]=[28.286398900000002, -16.796011808910734]
cd.loc[(cd.Country == 'Cayman Islands (the)'), ['Found Latitude', 'Found Longitude']]=[19.703182249999998, -79.9174627243246]
cd.loc[(cd.Country == 'Azores Islands'), ['Found Latitude', 'Found Longitude']]=[37.80855645, -25.473137391245295]
cd.loc[(cd.Country == 'American Samoa'), ['Found Latitude', 'Found Longitude']]=[-14.297124, -170.7131481]
cd.loc[(cd.Country == 'Wallis and Futuna'), ['Found Latitude', 'Found Longitude']]=[-13.289402, -176.204224]
cd.loc[(cd.Country == 'Virgin Island (British)'), ['Found Latitude', 'Found Longitude']]=[18.4024395, -64.5661642]
cd.loc[(cd.Country == 'Germany Dem Rep'), ['Found Latitude', 'Found Longitude']]=[52.290320, 13.099466]
cd.loc[(cd.Country == 'French Guiana'), ['Found Latitude', 'Found Longitude']]=[4.0039882, -52.999998]
cd.loc[(cd.Country == 'Sint Maarten (Dutch part)'), ['Found Latitude', 'Found Longitude']]=[18.0423736, -63.0549948]
cd.loc[(cd.Country == 'Saint Martin (French Part)'), ['Found Latitude', 'Found Longitude']]=[18.0814066, -63.0467131]
cd.loc[(cd.Country == 'Saint Barthélemy'), ['Found Latitude', 'Found Longitude']]=[17.9036287, -62.81152921501989]

In [10]:
cd['Found Latitude'].isnull().sum()

0

In [12]:
c1 = pd.merge(og, cd[['Country', 'Found Latitude', 'Found Longitude']], on='Country', how='left')

In [16]:
c1.columns = c1.columns.to_series().replace({'Found Latitude': 'Country Latitude', 'Found Longitude': 'Country Longitude'})

In [19]:
c1.head()

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Entry Criteria,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI,Country Latitude,Country Longitude
0,1900-9002-CPV,1900,9002,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,Cabo Verde,CPV,Western Africa,Africa,Countrywide,NaN,Famine,NaN,NaN,No,No,NaN,NaN,Km2,NaN,NaN,NaN,NaN,1900,NaN,NaN,1900,NaN,NaN,11000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389,16.000055,-24.008395
1,1900-9001-IND,1900,9001,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,India,IND,Southern Asia,Asia,Bengal,NaN,NaN,NaN,NaN,No,No,NaN,NaN,Km2,NaN,NaN,NaN,NaN,1900,NaN,NaN,1900,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389,22.351115,78.667743
2,1902-0012-GTM,1902,12,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Kill,Guatemala,GTM,Central America,Americas,"Quezaltenango, San Marcos",NaN,Tsunami/Tidal wave,NaN,NaN,NaN,NaN,NaN,8.0,Richter,14,-91,20:20,NaN,1902,4.0,18.0,1902,4.0,18.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,3.391845,15.585555,-90.345759
3,1902-0003-GTM,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,Guatemala,GTM,Central America,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1902,4.0,8.0,1902,4.0,8.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845,15.585555,-90.345759
4,1902-0010-GTM,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,Guatemala,GTM,Central America,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1902,10.0,24.0,1902,10.0,24.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845,15.585555,-90.345759


In [23]:
locFiles = ['FoundCoords/Locations_' + str(i).zfill(3) + '.csv' for i in range(100)]

In [25]:
ld = pd.concat((pd.read_csv(f, encoding='utf-16') for f in locFiles), ignore_index=True)

In [26]:
len(ld.index)

12515

In [27]:
ld.head()

,Unnamed: 0,Country,CountryAlpha2,Location,Count,Found Location,Found Latitude,Found Longitude
0,11899,United States,US,Texas,21,"Texas, United States",31.263890,-98.545612
1,11269,United States,US,California,18,"California, United States",36.701463,-118.755997
2,2753,China,CN,Yunnan province,17,"云南省, 中国",25.000000,102.000000
3,2611,China,CN,Sichuan province,17,"四川省, 中国",30.500000,102.500000
4,5114,India,IN,Uttar Pradesh,16,"Uttar Pradesh, India",27.130334,80.859666


In [28]:
ld['Found Latitude'].isnull().sum()

10220

In [29]:
c2 = pd.merge(c1, ld[['Country', 'Location', 'Found Latitude', 'Found Longitude']], on=['Country', 'Location'], how='left')

In [30]:
c2.columns = c2.columns.to_series().replace({'Found Latitude': 'Location Latitude', 'Found Longitude': 'Location Longitude'})

In [34]:
c2.head()

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Entry Criteria,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI,Country Latitude,Country Longitude,Location Latitude,Location Longitude
0,1900-9002-CPV,1900,9002,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,Cabo Verde,CPV,Western Africa,Africa,Countrywide,NaN,Famine,NaN,NaN,No,No,NaN,NaN,Km2,NaN,NaN,NaN,NaN,1900,NaN,NaN,1900,NaN,NaN,11000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389,16.000055,-24.008395,NaN,NaN
1,1900-9001-IND,1900,9001,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,India,IND,Southern Asia,Asia,Bengal,NaN,NaN,NaN,NaN,No,No,NaN,NaN,Km2,NaN,NaN,NaN,NaN,1900,NaN,NaN,1900,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389,22.351115,78.667743,NaN,NaN
2,1902-0012-GTM,1902,12,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Kill,Guatemala,GTM,Central America,Americas,"Quezaltenango, San Marcos",NaN,Tsunami/Tidal wave,NaN,NaN,NaN,NaN,NaN,8.0,Richter,14,-91,20:20,NaN,1902,4.0,18.0,1902,4.0,18.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,3.391845,15.585555,-90.345759,NaN,NaN
3,1902-0003-GTM,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,Guatemala,GTM,Central America,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1902,4.0,8.0,1902,4.0,8.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845,15.585555,-90.345759,NaN,NaN
4,1902-0010-GTM,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,Guatemala,GTM,Central America,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1902,10.0,24.0,1902,10.0,24.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845,15.585555,-90.345759,NaN,NaN


In [35]:
c2.to_csv('FoundCoords/Combined.csv', encoding='utf-16')